In [3]:
import os,sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

from util import *

In [5]:
from Functions.main import labelledmesh_procedures,graph_procedures,MSII_chaineoperatoire_procedures
from Classes.Graph.GraphPlotting import GraphEvaluation

import csv

def import_eval_links (path,id,link_paras):

    with open(''.join([path,id,link_paras,'.csv']), mode='r') as infile:
        reader = csv.reader(infile)
        next(reader, None)  # skip the headers
        eval = dict((tuple([int(e) for e in rows[0].split(' ')]),int(rows[1])) for rows in reader)

    return eval

def export_cleaned_links (path,id,link_paras,edges):

    with open(''.join([path,id,link_paras,'.csv']), mode='w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
                            quotechar='|')
        writer.writerow(['source','target'])
        for e in edges:
            print([str(e[0]),str(e[1])])
            writer.writerow([str(e[0]),str(e[1])])


In [13]:
exp_path =              'exp_path/'
id =                    '207'  # 
# id = 'RF.c_49'
preprocessed =          '_GMOCF_r1.00_n4_v256.volume'#'_GMOCF_gt_labels' #'_GMOCF'
labelname =             '_gt_labels'#'_Kmeans_labels'#
graphname =             'max'#'normal' # 'min' # 'max'# 'normal'
diameter =              1
n_rad =                 4    
radius_scale =          0.5
circumference_scale =   1
parameters =            ['quality']


path = '/home/linsel/Documents/PhD/Data/Fumane/V1.0.0_Linsel_et_al_2023/Graph/CO/{}/data/'.format(id)
# path = '/home/linsel/Documents/PhD/Data/Fumane/V1.0.0_Linsel_et_al_2023/Graph/CO/{}/PCA_quality/'.format(id)


kwargs = {'path':           path,
          'id':             id,
          'preprocessed':   preprocessed,
          'labelname':      labelname,
          'exp_path':       exp_path,
          'graphname':      graphname,
          'diameter':       diameter,
          'parameters':     parameters,
          'n_rad':          n_rad
          }


In [14]:
def graph_evaluation_procedure (obj,**kwargs):

    # LabelledMesh init
    path = kwargs ['path'] 
    id = kwargs ['id']
    preprocessed = kwargs ['preprocessed']
    labelname = kwargs ['labelname']
    exp_path = kwargs ['exp_path'] 
    eval = kwargs ['eval']
    para_name = kwargs ['para_name']

    # scales
    radius_scale = kwargs ['radius_scale'] 
    circumference_scale = kwargs ['circumference_scale'] 

    # Data import 
    obj.load_labelled_mesh(path,id,preprocessed,labelname,exp_path) 
    obj.extract_ridges()

    # get polylines
    obj.polyline_to_nx()
    obj.polygraphs_to_polylines()

    # create node coordinates
    obj.get_centroids()
    # obj.get_NNs()
    # obj.get_NNs()

    # prepare for creating MSII1D_Pline object 
    obj.create_normals_vertices()
    obj.create_dict_mesh_info()
    obj.prepare_polyline()

    # prepare for get ridges and for scar-ridge-graph model and leading to chaine-operatoire 
    obj.prep_ridges()
    obj.polineline_segmenting()
    obj.segment_to_ridgegraph()

    obj.create_undirected_model(radius_scale,circumference_scale)

    obj.create_graph_model_selected_labels (eval)

    obj.export_graph_model_labels(obj.vert_dict, para_name)


def graph_procedures (method,**kwargs):

    GE = GraphEvaluation ()

    procedures = {'graph_evaluation':graph_evaluation_procedure}

    func = procedures.get(method)

    func(GE,**kwargs)

    return GE    


def graph_procedures (method,**kwargs):

    GE = GraphEvaluation ()

    procedures = {'graph_evaluation':graph_evaluation_procedure}

    func = procedures.get(method)

    func(GE,**kwargs)

    return GE    

# 1241.2 (Robertinum)
para_name =  'betweenness'# 'degree'#'area'#
direction =  'greater'#'greater'#'greater'#
para_acc =  '0.72'# '0.71'# '0.78' # 

para_name = 'area'#'betweenness'# 'degree'#
direction =  'greater'#'greater'#'greater'#
para_acc =  '0.78'#'0.78'#'0.84'#

link_paras = '_' + '_'.join([para_name,direction, 'acc-{}'.format(para_acc),'links'])

# 207 (GdF)
# link_paras = '_area_greater_acc-0.84_eval_links'
# link_paras ='_betweeness_greater_acc-0.79_eval_links'
# link_paras ='_degree_greater_acc-0.78_eval_links'

# RF.c_49 (GdF)
# link_paras = '_area_less_acc-0.52_eval_links'
# link_paras ='_betweeness_less_acc-0.53_eval_links'
# link_paras ='_degree_less_acc-0.49_eval_links'

# link_paras ='_betweeness_greater_acc-0.72_eval_links'
# link_paras ='_degree_greater_acc-0.71_eval_links'

link_paras = '_rank_less_acc-1.0_links'

link_paras = '_betweeness_greater_acc-0.79_links'

eval = import_eval_links (path,id,link_paras)    

kwargs = {'path':path,
          'id':id,
          'preprocessed':preprocessed,
          'labelname':'_gt_labels',
          'exp_path':exp_path,
          'graphname':'normal',
          'radius_scale':radius_scale,
          'circumference_scale':circumference_scale,
          'eval':eval,
          'para_name': para_name
          }

G1 = graph_procedures('graph_evaluation', **kwargs)          